<a href="https://colab.research.google.com/github/atbender/training-h2ia/blob/main/sliding_puzzle_a_star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Sliding Puzzle Using Informed Search: A*

#### Author: Alexandre Thurow Bender

In this notebook, we solve the sliding puzzle game using **A\* Search**. The objective of this report is to compare the impact of different heuristics in guiding the A* Search.

In [1]:
import numpy as np
import random
import pandas as pd

We model the problem state space using Nodes. Each Node stores the state, its parent Node, and the directional move that generated the Node's state.

Storing the state is necessary for modelling the board configuration of the puzzle, and for this we use a 2d numpy array.

It is relevant to note the answer to the sliding puzzle problem is not the final state (as it is previously known), but the sequence of movements that take us from the initial configuration to the final state. Storing the parent Node is necessary in order to retrieve the path to the solution, as this implementation does not use an explicit tree data structure but instead uses lists and queues to simulate the tree search traversal.

The Node structure also saves the directional move that generated the state, as this makes it easier to visualize and rebuild the movement sequence solution.

In addition to these attributes, Nodes now also save their heuristic cost, as this is necessary for the information aspect of the search.

We also implement the less than dunder method, necessary for ordening objects in heaps. The implementarion itself does not matter, since we use tuple ordering in the priority queue (nevertheless, the less than method needs to exist).

In [2]:
class Node:
    def __init__(self, rows, columns, level=None, parent=None):
        self.parent = parent
        self.move = None
        self.cost = 0
        if level is not None:
            self.level = level
            return
        self.level = self.create_level(rows, columns)
        

    def create_level(self, rows, columns):
        x = np.arange(0, rows*columns, dtype=int)
        np.random.shuffle(x)
        return x.reshape(rows, columns)

    def __repr__(self):
        return str(self.level)

    def __str__(self):
        return str(self.level)

    def display(self):
        return pd.DataFrame(self.level).style.hide_index().hide_columns()
    
    def __lt__(self, other):
        return True

Generating a random board state is trivial, but it is important to realize not all board states are solvable. We are not interested in generating unsolvable board states as it would hinder the comparison between uninformed searches (or any search, for that matter). 

It would be frustrating if the sliding puzzle game would generate unsolvable states for human players, therefore I assume that would also be the case for search methods.

In [3]:
my_level = Node(3, 5)
my_level.display()

4,1,3,11,2
6,0,10,12,5
9,13,14,8,7


Generating a solvable board instance of the sliding puzzle is harder than checking if a given instance is solvable. For this reason we keep re-generating random instances until the resulting configuration instance is solvable.

Checking for solvability takes into account the parity of the board width, the parity of the total number of inversions in the state map, and the parity of the empty square's row index.

In [4]:
def is_level_solvable(Node):
    level = Node.level
    if is_even(len(level[0])):
        print("level width is even")
        return check_parity_even(level)
    print("level width is odd")
    return check_parity_odd(level)

def check_parity_even(level):
    index_from_below = len(level) - np.where(level == 0)[0].item()
    if is_even(index_from_below):
        # If the grid width is even, and the blank is on an even row counting from the
        # bottom (second-last, fourth-last etc), then the number of inversions in a solvable
        # situation is odd.
        print("index from below is even")
        return not is_even(count_inversions(level))

    # If the grid width is even, and the blank is on an odd row counting from the bottom
    # (last, third-last, fifth-last etc) then the number of inversions in a solvable situation
    # is even.
    print("index from below is odd")
    return is_even(count_inversions(level))

def check_parity_odd(level):
    # If the grid width is odd, then the number of inversions in a solvable situation is even.
    return is_even(count_inversions(level))
    
def count_inversions(level):
    level = level.flatten()
    inversions = 0

    for i in range(0, len(level)):
        for j in range(i + 1, len(level)):
            if level[i] == 0 or level[j] == 0:
                continue
            if level[i] > level[j]:
                inversions += 1
    print(f"level has {inversions} inversions")
    return inversions

def is_even(number):
    return not number % 2

In [5]:
my_level = Node(2, 3)
print(is_level_solvable(my_level))
my_level.display()

level width is odd
level has 4 inversions
True


4,0,1
3,2,5


Below we generate some specific instances to test our check for solvability.

In [6]:
test = Node(2, 3, level=np.array([[5,4,3], [2,1,0]]))
print(is_level_solvable(test))
test.display()

level width is odd
level has 10 inversions
True


5,4,3
2,1,0


In [7]:
test2 = Node(2, 4, np.array([[7,6,5,0], [4,3,2,1]]))
print(is_level_solvable(test2))
test2.level

level width is even
index from below is even
level has 21 inversions
True


array([[7, 6, 5, 0],
       [4, 3, 2, 1]])

In [8]:
test3 = Node(2, 4, level=np.array([[7,6,5,4], [3,2,1,0]]))
print(is_level_solvable(test3))
test3.display()

level width is even
index from below is odd
level has 21 inversions
False


7,6,5,4
3,2,1,0


Creating solvable state instances can be also be done by shuffling a solution with the possible moves. This is arguably more computational intensive, but would garantee the correct initial state generation as we would be restricted to traversing the problem state space.

In [9]:
def create_solvable_level(rows, columns):
    level = Node(rows, columns)
    while not is_level_solvable(level):
        level = Node(rows, columns)
    return level

In [10]:
is_level_solvable(create_solvable_level(2, 3))

level width is odd
level has 7 inversions
level width is odd
level has 5 inversions
level width is odd
level has 6 inversions
level width is odd
level has 6 inversions


True

In [11]:
level = Node(2, 3)

In [12]:
def get_zero_index(level):
    np_index = np.where(level == 0)
    return np_index[0][0], np_index[1][0]

In order to generate neighboring states we must represent the game rules in child state generation. This is done by modelling the possible moves with respect to column and row boundaries.

In [13]:
def generate_neighbors(node):
    zero = get_zero_index(node.level)
    neighbors = []

    # upper bound
    if zero[0] > 0:
        neighbors.append(slide_up(node, zero))
    # left bound
    if zero[1] > 0:
        neighbors.append(slide_left(node, zero))
    # lower bound
    if zero[0] < len(node.level)-1:
        neighbors.append(slide_down(node, zero))
    # right_bound
    if zero[1] < len(node.level[0])-1:
        neighbors.append(slide_right(node, zero))

    return neighbors

The piece slide is implemented with respect to the blank square (zero). Each directional move swaps the blank square with the neighboring number. This is also the moment where the parents and directional moves are saved by the Nodes.

In [14]:
def slide_up(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node.level)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]-1][zero[1]]
    level[zero[0]-1][zero[1]] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = "^"
    return generated_node

def slide_left(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]][zero[1]-1]
    level[zero[0]][zero[1]-1] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = "<"
    return generated_node

def slide_down(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]+1][zero[1]]
    level[zero[0]+1][zero[1]] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = "v"
    return generated_node

def slide_right(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]][zero[1]+1]
    level[zero[0]][zero[1]+1] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = ">"
    return generated_node

In [15]:
level = create_solvable_level(2, 3)
level.display()

level width is odd
level has 7 inversions
level width is odd
level has 3 inversions
level width is odd
level has 4 inversions


4,0,1
3,2,5


In [16]:
neighbors = generate_neighbors(level)

In [17]:
for item in neighbors:
    print(item, "\n")

[[0 4 1]
 [3 2 5]] 

[[4 2 1]
 [3 0 5]] 

[[4 1 0]
 [3 2 5]] 



To implement any search, we must set up a way of evaluating the states. In uninformed searches, our evaluation is simply checking whether a state is the final state.

In [18]:
def is_solution(node):
    level = node.level
    level = level.flatten()
    if level[-1] != 0:
        return False
    level = level[:-1]
    return np.all(level[:-1] < level[1:])

In [19]:
node = Node(0, 0, level=np.array([[1,2,3], [4,5,0]]))
is_solution(node)

True

In [20]:
node.level

array([[1, 2, 3],
       [4, 5, 0]])

In [21]:
def generate_solution(rows, columns):
    x = np.arange(1, rows*columns+1, dtype=int)
    x[-1] = 0
    return x.reshape(rows, columns)

generate_solution(3,4)

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11,  0]])

Contrary to uninformed searches, informed searches use domain problem knowledge to guide the search process. Tipically, such knowledge is encoded using heuristics.

Heuristics are state evaluation functions. They are used to estimate how close to the solution a given state is. This information usually comes in the form of a cost (i.e. number elements in the wrong position of the sliding puzzle level). The reason searches often use estimates is because evaluating the real cost of a state would ultimately entail in solving the problem itself.

Designing heuristics is a complicated engineering task. Their potential effectiveness is strongly correlated with how the information is represented in states.

In this report we implement two heuristics: wrong position count and solution difference.

Wrong position count is quite intuitive and consists of simply counting the wrongly positioned element instances. Note this heuristic approach abstracts how wrong an element is. In fact, the degrees of state costs are limited to the number of level elements.

Solution difference can be understood as a mean absolute error of the elements in a level with regards to the solution. In addition to taking into account wrong positions, this approach also models locality and how wrong a given position is.

In [22]:
def count_wrong_positions(node, solution):
    flatten_node = node.level.flatten()
    solution = solution.flatten()
    return np.count_nonzero(flatten_node!=solution)

def difference(node, solution):
    flatten_node = node.level.flatten()
    solution = solution.flatten()
    return sum([abs(i - j) for i, j in zip(flatten_node, solution)])

def calculate_cost(node, solution, heuristic=count_wrong_positions):
    return heuristic(node, solution)

In [23]:
level = create_solvable_level(3, 3)
solution = generate_solution(3, 3)
level.display()

level width is odd
level has 15 inversions
level width is odd
level has 17 inversions
level width is odd
level has 10 inversions


2,0,4
7,3,1
6,8,5


In [24]:
calculate_cost(level, solution, count_wrong_positions)

8

In [25]:
calculate_cost(level, solution, difference)

20

A* seach expands elements with lower cost of their heuristic + path value. The path value in our case is simply the number of moves necessary to reach a given state, while the heuristic can be chosen from the implemented heuristics described above. To expand the nodes we use a priority queue implemented as a heap. We also use a dictionary structure to store the path costs with respect to each node.

After a solution is found, the algorithm keeps expanding nodes until their path cost is bigger than the solution path. This garantees no  better alternative solutions exists, ensuring search optimality.

In [26]:
import heapq

def a_star(start_node, heuristic):
    start_node.cost = calculate_cost(start_node, solution, heuristic)
    frontier = [(start_node.cost + 0, start_node)]
    in_frontier = set()
    in_frontier.add(hash(str(start_node)))
    cost_so_far = {}
    cost_so_far[hash(str(start_node.level))] = 0
    solution_cost = None
    solution_node = None
    i = 1
    
    while True:
        if not frontier:
            break
            return "failure"

        node = heapq.heappop(frontier)[1]
        #in_frontier.remove(hash(str(node)))
        print(f"expanded {i} nodes")
        i += 1

        if is_solution(node):
            print("solution path found")
            if solution_cost is None or cost_so_far[hash(str(node.level))] < solution_cost:
                solution_cost = cost_so_far[hash(str(node.level))]
                print(f"better solution found in \n{node.level}")
                solution_node = node

        for neighbor in generate_neighbors(node):
            neighbor.cost = calculate_cost(neighbor, solution, heuristic)
            new_cost = cost_so_far[hash(str(node.level))] + 1

            if solution_cost is not None and new_cost > solution_cost:
                continue

            if hash(str(neighbor.level)) not in cost_so_far.keys() or new_cost < cost_so_far[hash(str(neighbor.level))]:
                cost_so_far[hash(str(neighbor.level))] = new_cost
                neighbor.parent = node
                heapq.heappush(frontier, (neighbor.cost + new_cost, neighbor))
                
    print("retrieving best solution")
    retrieve_path(solution_node)
        

Retrieving the path and move sequence can be accomplished by storing the parent hierarchy bottom-up from the solution found and inverting it.

In [27]:
def retrieve_path(state):
    path = []
    move_list = []

    while state is not None:
        path.append(state)
        if state.move is not None:
            move_list.append(state.move)
        state = state.parent
    
    print("---path is:---")

    move_list.insert(0, "#")
    print(move_list[::-1])

    for state, move in zip(path[::-1], move_list[::-1]):
        print(f"\n{state.level}")
        print(move)
    
    print(len(move_list)-1)
    return path

In [28]:
level = create_solvable_level(2, 2)
solution = generate_solution(2, 2)
level.display()

level width is even
index from below is odd
level has 2 inversions


2,3
0,1


In [29]:
a_star(level, count_wrong_positions)

expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
expanded 4 nodes
expanded 5 nodes
expanded 6 nodes
expanded 7 nodes
expanded 8 nodes
expanded 9 nodes
solution path found
better solution found in 
[[1 2]
 [3 0]]
expanded 10 nodes
expanded 11 nodes
retrieving best solution
---path is:---
['>', '^', '<', 'v', '>', '#']

[[2 3]
 [0 1]]
>

[[2 3]
 [1 0]]
^

[[2 0]
 [1 3]]
<

[[0 2]
 [1 3]]
v

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
5


In [30]:
!pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
%load_ext memory_profiler

Now we profile the memory and time costs of A* with the count wrong positions heuristic.

In [32]:
level = create_solvable_level(2, 2)
level.display()

level width is even
index from below is odd
level has 1 inversions
level width is even
index from below is even
level has 1 inversions


0,2
1,3


In [33]:
%%timeit
%memit a_star(level, count_wrong_positions)

expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution found in 
[[1 2]
 [3 0]]
expanded 4 nodes
expanded 5 nodes
retrieving best solution
---path is:---
['v', '>', '#']

[[0 2]
 [1 3]]
v

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 143.62 MiB, increment: 0.09 MiB
expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution found in 
[[1 2]
 [3 0]]
expanded 4 nodes
expanded 5 nodes
retrieving best solution
---path is:---
['v', '>', '#']

[[0 2]
 [1 3]]
v

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 143.73 MiB, increment: 0.00 MiB
expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution found in 
[[1 2]
 [3 0]]
expanded 4 nodes
expanded 5 nodes
retrieving best solution
---path is:---
['v', '>', '#']

[[0 2]
 [1 3]]
v

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 143.80 MiB, increment: 0.00 MiB
expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution foun

This is a small instance, so the time costs are mostly negligible. But do note finding the shortest solution for the n-sliding puzzle is NP-Hard.

In [34]:
level = create_solvable_level(2, 3)
level.display()

level width is odd
level has 2 inversions


3,1,2
4,5,0


In [35]:
%%timeit
%memit a_star(level, count_wrong_positions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
expanded 4 nodes
expanded 5 nodes
expanded 6 nodes
expanded 7 nodes
expanded 8 nodes
expanded 9 nodes
expanded 10 nodes
expanded 11 nodes
expanded 12 nodes
expanded 13 nodes
expanded 14 nodes
expanded 15 nodes
expanded 16 nodes
expanded 17 nodes
expanded 18 nodes
expanded 19 nodes
expanded 20 nodes
expanded 21 nodes
expanded 22 nodes
expanded 23 nodes
expanded 24 nodes
expanded 25 nodes
expanded 26 nodes
expanded 27 nodes
expanded 28 nodes
expanded 29 nodes
expanded 30 nodes
expanded 31 nodes
expanded 32 nodes
expanded 33 nodes
expanded 34 nodes
expanded 35 nodes
expanded 36 nodes
expanded 37 nodes
expanded 38 nodes
expanded 39 nodes
expanded 40 nodes
expanded 41 nodes
expanded 42 nodes
expanded 43 nodes
expanded 44 nodes
expanded 45 nodes
expanded 46 nodes
expanded 47 nodes
expanded 48 nodes
expanded 49 nodes
expanded 50 nodes
expanded 51 nodes
expanded 52 nodes
expanded 53 nodes
expanded 54 nodes
expanded 55 nodes
expanded 56 nodes
e

In [36]:
level = create_solvable_level(2, 4)
level.display()

level width is even
index from below is odd
level has 11 inversions
level width is even
index from below is even
level has 9 inversions


6,2,0,1
5,4,3,7


In [37]:
%%timeit
%memit a_star(level, count_wrong_positions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


Streaming output truncated to the last 5000 lines.
expanded 5480 nodes
expanded 5481 nodes
expanded 5482 nodes
expanded 5483 nodes
expanded 5484 nodes
expanded 5485 nodes
expanded 5486 nodes
expanded 5487 nodes
expanded 5488 nodes
expanded 5489 nodes
expanded 5490 nodes
expanded 5491 nodes
expanded 5492 nodes
expanded 5493 nodes
expanded 5494 nodes
expanded 5495 nodes
expanded 5496 nodes
expanded 5497 nodes
expanded 5498 nodes
expanded 5499 nodes
expanded 5500 nodes
expanded 5501 nodes
expanded 5502 nodes
expanded 5503 nodes
expanded 5504 nodes
expanded 5505 nodes
expanded 5506 nodes
expanded 5507 nodes
expanded 5508 nodes
expanded 5509 nodes
expanded 5510 nodes
expanded 5511 nodes
expanded 5512 nodes
expanded 5513 nodes
expanded 5514 nodes
expanded 5515 nodes
expanded 5516 nodes
expanded 5517 nodes
expanded 5518 nodes
expanded 5519 nodes
expanded 5520 nodes
expanded 5521 nodes
expanded 5522 nodes
expanded 5523 nodes
expanded 5524 nodes
expanded 5525 nodes
expanded 5526 nodes
expanded 

In [38]:
level = create_solvable_level(3, 3)
level.display()

level width is odd
level has 9 inversions
level width is odd
level has 13 inversions
level width is odd
level has 7 inversions
level width is odd
level has 13 inversions
level width is odd
level has 8 inversions


1,4,6
3,2,8
5,7,0


In [39]:
%%timeit
%memit a_star(level, count_wrong_positions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.


Streaming output truncated to the last 5000 lines.
expanded 105379 nodes
expanded 105380 nodes
expanded 105381 nodes
expanded 105382 nodes
expanded 105383 nodes
expanded 105384 nodes
expanded 105385 nodes
expanded 105386 nodes
expanded 105387 nodes
expanded 105388 nodes
expanded 105389 nodes
expanded 105390 nodes
expanded 105391 nodes
expanded 105392 nodes
expanded 105393 nodes
expanded 105394 nodes
expanded 105395 nodes
expanded 105396 nodes
expanded 105397 nodes
expanded 105398 nodes
expanded 105399 nodes
expanded 105400 nodes
expanded 105401 nodes
expanded 105402 nodes
expanded 105403 nodes
expanded 105404 nodes
expanded 105405 nodes
expanded 105406 nodes
expanded 105407 nodes
expanded 105408 nodes
expanded 105409 nodes
expanded 105410 nodes
expanded 105411 nodes
expanded 105412 nodes
expanded 105413 nodes
expanded 105414 nodes
expanded 105415 nodes
expanded 105416 nodes
expanded 105417 nodes
expanded 105418 nodes
expanded 105419 nodes
expanded 105420 nodes
expanded 105421 nodes
exp

Observe the time difference from solving a 2x3 instance to solving a 3x3 instance. Now we evaluate the difference heuristic.

In [40]:
level = create_solvable_level(2, 2)
level.display()

level width is even
index from below is even
level has 1 inversions


0,2
1,3


In [41]:
%%timeit
%memit a_star(level, difference)

expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution found in 
[[1 2]
 [3 0]]
expanded 4 nodes
expanded 5 nodes
retrieving best solution
---path is:---
['v', '>', '#']

[[0 2]
 [1 3]]
v

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 197.28 MiB, increment: 0.00 MiB
expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution found in 
[[1 2]
 [3 0]]
expanded 4 nodes
expanded 5 nodes
retrieving best solution
---path is:---
['v', '>', '#']

[[0 2]
 [1 3]]
v

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 197.28 MiB, increment: 0.00 MiB
expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution found in 
[[1 2]
 [3 0]]
expanded 4 nodes
expanded 5 nodes
retrieving best solution
---path is:---
['v', '>', '#']

[[0 2]
 [1 3]]
v

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 197.28 MiB, increment: 0.00 MiB
expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
solution path found
better solution foun

In [42]:
level = create_solvable_level(2, 3)
level.display()

level width is odd
level has 8 inversions


5,4,2
1,0,3


In [43]:
%%timeit
%memit a_star(level, difference)

expanded 1 nodes
expanded 2 nodes
expanded 3 nodes
expanded 4 nodes
expanded 5 nodes
expanded 6 nodes
expanded 7 nodes
expanded 8 nodes
expanded 9 nodes
expanded 10 nodes
expanded 11 nodes
expanded 12 nodes
expanded 13 nodes
expanded 14 nodes
expanded 15 nodes
expanded 16 nodes
expanded 17 nodes
expanded 18 nodes
expanded 19 nodes
expanded 20 nodes
expanded 21 nodes
expanded 22 nodes
expanded 23 nodes
expanded 24 nodes
expanded 25 nodes
expanded 26 nodes
solution path found
better solution found in 
[[1 2 3]
 [4 5 0]]
expanded 27 nodes
expanded 28 nodes
expanded 29 nodes
expanded 30 nodes
expanded 31 nodes
expanded 32 nodes
expanded 33 nodes
expanded 34 nodes
expanded 35 nodes
expanded 36 nodes
expanded 37 nodes
expanded 38 nodes
expanded 39 nodes
expanded 40 nodes
expanded 41 nodes
expanded 42 nodes
expanded 43 nodes
expanded 44 nodes
expanded 45 nodes
expanded 46 nodes
expanded 47 nodes
expanded 48 nodes
expanded 49 nodes
expanded 50 nodes
retrieving best solution
---path is:---
['^'

In [44]:
level = create_solvable_level(2, 4)
level.display()

level width is even
index from below is odd
level has 5 inversions
level width is even
index from below is odd
level has 12 inversions


1,6,7,5
0,3,2,4


In [45]:
%%timeit
%memit a_star(level, difference)

Streaming output truncated to the last 5000 lines.
expanded 2395 nodes
expanded 2396 nodes
expanded 2397 nodes
expanded 2398 nodes
expanded 2399 nodes
expanded 2400 nodes
expanded 2401 nodes
expanded 2402 nodes
expanded 2403 nodes
expanded 2404 nodes
expanded 2405 nodes
expanded 2406 nodes
expanded 2407 nodes
expanded 2408 nodes
expanded 2409 nodes
expanded 2410 nodes
expanded 2411 nodes
expanded 2412 nodes
expanded 2413 nodes
expanded 2414 nodes
expanded 2415 nodes
expanded 2416 nodes
expanded 2417 nodes
expanded 2418 nodes
expanded 2419 nodes
expanded 2420 nodes
expanded 2421 nodes
expanded 2422 nodes
expanded 2423 nodes
expanded 2424 nodes
expanded 2425 nodes
expanded 2426 nodes
expanded 2427 nodes
expanded 2428 nodes
expanded 2429 nodes
expanded 2430 nodes
expanded 2431 nodes
expanded 2432 nodes
expanded 2433 nodes
expanded 2434 nodes
expanded 2435 nodes
expanded 2436 nodes
expanded 2437 nodes
expanded 2438 nodes
expanded 2439 nodes
expanded 2440 nodes
expanded 2441 nodes
expanded 

In [46]:
level = create_solvable_level(3, 3)
level.display()

level width is odd
level has 15 inversions
level width is odd
level has 16 inversions


0,6,5
3,8,1
4,2,7


In [47]:
%%timeit
%memit a_star(level, difference)

Streaming output truncated to the last 5000 lines.
expanded 146645 nodes
expanded 146646 nodes
expanded 146647 nodes
expanded 146648 nodes
expanded 146649 nodes
expanded 146650 nodes
expanded 146651 nodes
expanded 146652 nodes
expanded 146653 nodes
expanded 146654 nodes
expanded 146655 nodes
expanded 146656 nodes
expanded 146657 nodes
expanded 146658 nodes
expanded 146659 nodes
expanded 146660 nodes
expanded 146661 nodes
expanded 146662 nodes
expanded 146663 nodes
expanded 146664 nodes
expanded 146665 nodes
expanded 146666 nodes
expanded 146667 nodes
expanded 146668 nodes
expanded 146669 nodes
expanded 146670 nodes
expanded 146671 nodes
expanded 146672 nodes
expanded 146673 nodes
expanded 146674 nodes
expanded 146675 nodes
expanded 146676 nodes
expanded 146677 nodes
expanded 146678 nodes
expanded 146679 nodes
expanded 146680 nodes
expanded 146681 nodes
expanded 146682 nodes
expanded 146683 nodes
expanded 146684 nodes
expanded 146685 nodes
expanded 146686 nodes
expanded 146687 nodes
exp

Comparing the two heuristics we see that the difference heuristic is somewhat faster than the count wrong positions heuristic. This is attributed to the greater positional information it includes. Even so, the difference heuristic calculation for a given state is slower. This means that it might be problematic in tasks where the branching factor is too great.

# Conclusion

In the report we explored A*, an informed search and two possible heuristics to solve the sliding puzzle problem.

A* has several advantages, including its completeness and optimality. Its path cost calculation can, however, be let astray by states of similar cost. This is particulaly true for states close to the solution.

The creation of heuristics is also an important (and difficult) part of informed searches, and heavily influences the search behavior.